<h1>Análisis Exploratorio de Datos (EDA) - Descripción General y Transformaciones

En este Notebook se llevará a cabo la descripción general de los datos, así como las transformaciones correspondientes para el EDA de las bases de datos de los sinietros de la ciudad de Buenos Aires, Argentina. Se dividirá en dos partes para asegurar el buen rendimiento del notebook. En la [segunda parte](EDA_PT2.ipynb) se realizarán el análisis utilizando herramientas gráficas.

<h2>Importaciones

In [165]:
import pandas as pd
import numpy as np

<h2>Funciones

Se crea función para conocer cuántos datos únicos hay en una columna de dataframe y el porcentaje que estos representan.

In [166]:
def dataPercentage(df, column):
    counts = df[column].value_counts()
    percentages = round(100 * counts / len(df), 2)
    dfResults = pd.DataFrame({
        "Cantidad":counts,
        "Porcentaje":percentages
    })
    return dfResults

Se crea función para saber el tipo de dato que hay en las funciones, incluyendo el porcentaje de nulos.

In [167]:
def dataType(df):
    dfDict = {
        "name": [], "data_type": [], "not_null_%": [], "null_%": [], "null": []
    }
    for column in df.columns:
        notNull = (df[column].count() / len(df)) * 100
        dfDict["name"].append(column)
        dfDict["data_type"].append(df[column].apply(type).unique())
        dfDict["not_null_%"].append(round(notNull, 2))
        dfDict["null_%"].append(round(100-notNull, 2))
        dfDict["null"].append(df[column].isnull().sum())
    dataTypeDf=pd.DataFrame(dfDict)
    return dataTypeDf

<h2>Homicidios - Hechos

<h3>Revisión Inicial

Se cargan el dataset y se crea el dataframe correspondiente.

In [168]:
homicidiosHechosDf = pd.read_excel('../data/homicidios.xlsx',sheet_name='HECHOS')

Empezamos analizando el dataframe de hechos, vemos que inicialmente hay un total de 696 registros. A primera instancia los datos se ven relativamente de buena calidad, se prevee poca necesidad de transformación.

In [169]:
homicidiosHechosDf.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [170]:
homicidiosHechosDf.shape[0]

696

Normalizamos los nombres de las columnas.

In [171]:
homicidiosHechosDf.columns = [x.lower() for x in homicidiosHechosDf.columns]
homicidiosHechosDf.columns = homicidiosHechosDf.columns.str.replace(' ', '_')
homicidiosHechosDf = homicidiosHechosDf.rename(columns={'id': 'id_hecho'})
homicidiosHechosDf.columns

Index(['id_hecho', 'n_victimas', 'fecha', 'aaaa', 'mm', 'dd', 'hora', 'hh',
       'lugar_del_hecho', 'tipo_de_calle', 'calle', 'altura', 'cruce',
       'dirección_normalizada', 'comuna', 'xy_(caba)', 'pos_x', 'pos_y',
       'participantes', 'victima', 'acusado'],
      dtype='object')

<h3>Revisión de Datos

<h4>Datos duplicados

Comprobamos que no existen filas duplicadas. Asímismo se considera relevante comprobar que no se repitan datos específicamente en la columna de 'id_hecho'.

In [172]:
homicidiosHechosDf[homicidiosHechosDf.duplicated()]

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado


In [173]:
homicidiosHechosDf[homicidiosHechosDf.duplicated(
    subset=['id_hecho'],
    keep=False
)]

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado


<h4>Datos Nulos

Aplicamos la función dataType para determinar la cantidad de valores nulos.

In [174]:
dataType(homicidiosHechosDf)

,name,data_type,not_null_%,null_%,null
0,id_hecho,[<class 'str'>],100.00,0.00,0
1,n_victimas,[<class 'int'>],100.00,0.00,0
2,fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.00,0.00,0
3,aaaa,[<class 'int'>],100.00,0.00,0
4,mm,[<class 'int'>],100.00,0.00,0
5,dd,[<class 'int'>],100.00,0.00,0
6,hora,"[<class 'datetime.time'>, <class 'str'>, <clas...",100.00,0.00,0
7,hh,"[<class 'int'>, <class 'str'>]",100.00,0.00,0
8,lugar_del_hecho,[<class 'str'>],100.00,0.00,0
9,tipo_de_calle,[<class 'str'>],100.00,0.00,0


La primera columna con datos faltantes es la de 'calle' con 1 dato nulo representando el 0.14% de los datos. Al no tener la información para completar los datos, se decide imputar el dato nulo como 'SD' indicando que es Sin Dato.

In [175]:
homicidiosHechosDf[homicidiosHechosDf['calle'].isnull()]

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD


In [176]:
homicidiosHechosDf['calle'] = homicidiosHechosDf['calle'].fillna('SD')
homicidiosHechosDf[homicidiosHechosDf['calle'] == 'SD']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD


Pasando a las columnas de 'altura' y 'cruce', estas tienen 567 y 171 datos faltantes, representando el 81.47% y el 24.57% de los datos respectivamente. Vemos que tienen una relación interesante ya que son mutuamente excluyentes. Cuando un siniestro ocurre dentro de una calle se anota la altura a la cual sucedió, en esos casos no hay cruzamiento. Por el contrario cuando ocurre en un cruzamiento, este se anota y queda vacío el campo de altura. Podemos observar que no hay instancias en donde ambos datos se encuentren vacíos, por lo que no existe necesidad de eliminar los nulos.

In [177]:
homicidiosHechosDf[homicidiosHechosDf['altura'].isnull()].head(5)

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS
5,2016-0008,1,2016-01-24,2016,1,24,18:30:00,18,AV 27 DE FEBRERO Y AV ESCALADA,AVENIDA,...,NaN,ESCALADA AV.,27 DE FEBRERO AV. y ESCALADA AV.,8,Point (101721.59002217 93844.25656649),-58.44451316,-34.68475866,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO


In [178]:
homicidiosHechosDf[homicidiosHechosDf['cruce'].isnull()].head(5)

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
9,2016-0013,1,2016-02-10,2016,2,10,11:30:00,11,AV ENTRE RIOS 1366,AVENIDA,...,1366.0,NaN,ENTRE RIOS AV. 1366,1,Point (106616.41069662 100496.44662323),-58.39114932,-34.62477387,PEATON-AUTO,PEATON,AUTO
14,2016-0020,1,2016-02-17,2016,2,17,16:00:00,16,SUIPACHA 156,CALLE,...,156.0,NaN,SUIPACHA 156,1,Point (107684.31807562 102519.17333314),-58.37952223,-34.60653282,PEATON-AUTO,PEATON,AUTO
33,2016-0047,1,2016-04-15,2016,4,15,05:30:00,5,LIMA 1483,CALLE,...,1483.0,NaN,LIMA 1483,1,Point (107412.19098904 100391.02557777),-58.38247061,-34.62571878,PEATON-PASAJEROS,PEATON,PASAJEROS
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,...,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,SD,SD


In [179]:
# 0 casos donde ambos contengan datos.
homicidiosHechosDf[homicidiosHechosDf[['altura', 'cruce']].notna().all(axis=1)]

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado


Ahora bien, existen casos en donde ambos son nulos. Creamos un DataFrame para analizar estos datos.

In [180]:
alturaCruceNullDf = homicidiosHechosDf[
    homicidiosHechosDf[['altura','cruce']].isnull().all(axis=1)
]
alturaCruceNullDf.head()

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,...,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,SD,SD
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,...,NaN,NaN,NaN,13,Point (. .),.,.,MOTO-SD,MOTO,SD
64,2016-0087,1,2016-07-02,2016,7,3,00:10:00,0,AUTOPISTA 1 SUR PTE ARTURO FRONDIZI Y AV CASEROS,AUTOPISTA,...,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI y C...,1,Point (107762.62066736 100018.90176187),-58.37864583,-34.62907067,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
71,2016-0096,1,2016-07-25,2016,7,25,07:00:00,7,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA,...,NaN,NaN,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,8,Point (. .),-58.47433193007387,-34.66684950051973,MOTO-CARGAS,MOTO,CARGAS
81,2016-0107,1,2016-08-20,2016,8,20,08:22:00,8,AUTOPISTA 9 DE JULIO SUR ALT AV MENDOZA,AUTOPISTA,...,NaN,NaN,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI y D...,4,Point (108408.31858686 97219.56218484),-58.37157668,-34.65429986,MOTO-AUTO,MOTO,AUTO


Utilizando la función dataPercentage observamos la cantidad de datos únicos de la columna 'tipo_de_calle', vemos que existe una predominancia del dato 'AUTOPISTA' con el 92.86%, lo cual lleva a pensar que puede ser una caractarística de esa clasificación que las columnas de 'altura' y 'cruce' queden nulas.

In [181]:
dataPercentage(alturaCruceNullDf,'tipo_de_calle')

,Cantidad,Porcentaje
tipo_de_calle,,
AUTOPISTA,39,92.86
CALLE,2,4.76
GRAL PAZ,1,2.38


Sin embargo, vale la pena revisar el motivo de los tipos de calle 'CALLE' y 'GRAL PAZ' que cuentan con datos nulos.

In [182]:
alturaCruceNullDf[alturaCruceNullDf['tipo_de_calle'] == 'CALLE']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD
559,2020-0039,1,2020-09-01,2020,9,1,19:17:42,19,MURGUIONDO 2700,CALLE,...,NaN,NaN,MURGUIONDO,9,Point (. .),.,.,PEATON-CARGAS,PEATON,CARGAS


In [183]:
alturaCruceNullDf[alturaCruceNullDf['tipo_de_calle'] == 'GRAL PAZ']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,...,NaN,NaN,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS


Además de el hecho con id '2016-0151' que ya se había identificado previamente, nos encontramos con dos casos adicionales con datos de localización desconocidos. A pesar de que podemos considerarlos como outliers, puede intuirse que se trata de casos donde se dio el siniestro, sin embargo, las autoridades no fueron capaces de recabar los datos necesarios. Se toma la decisión de mantener estos casos ya que los datos nulos no afectan la relevancia del siniestro para métricas como el número de siniestros totales, participantes en los siniestros, etc.

Ahora veremos los casos en donde al menos una de las dos variables no es nula. Guardamos los datos donde al menos una de las dos no es nula y repetimos el proceso.

In [184]:
alturaCruceNotNullDf = homicidiosHechosDf[
    homicidiosHechosDf[['altura','cruce']].notna().any(axis=1)
]
alturaCruceNotNullDf.head()

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,altura,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


In [185]:
dataPercentage(alturaCruceNotNullDf,'tipo_de_calle')

,Cantidad,Porcentaje
tipo_de_calle,,
AVENIDA,429,65.60
CALLE,134,20.49
GRAL PAZ,64,9.79
AUTOPISTA,27,4.13


En este caso llaman la atención los casos los que el tipo de calle es 'AUTOPISTA' ya que contrasta con el resultado anterior, esta vez representando solo el 4.13% de los datos. Al revisar la distribución de los datos nulos vemos que son todos del tipo 'altura'. Esto nos confirma que efectivamente sí existen casos donde haya cruces con autopistas, sin embargo, son la minoría.

In [186]:
alturaCruceNotNullDf[
    alturaCruceNotNullDf['tipo_de_calle'] == 'AUTOPISTA'
][['altura','cruce']].count()

altura     0
cruce     27
dtype: int64

Se decide eliminar la columna de 'altura' ya que no proporciona datos relevantes para el análisis. Para la columna de 'cruce' se realiza la transformación a booleano teniendo True cuando se trata de un cruce y False cuando no.

In [187]:
del homicidiosHechosDf['altura']

In [188]:
homicidiosHechosDf['cruce'] = np.where(
    homicidiosHechosDf['cruce'].notnull(),
    True,
    False
)

La tercer columna con valores faltantes es 'dirección_normalizada' tiene solo 8 datos nulos, lo que representa el 1.15% de los datos de la columna. Se orservan esos registros a continuación.

In [189]:
homicidiosHechosDf[homicidiosHechosDf['dirección_normalizada'].isnull()]

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,calle,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,NaN,13,Point (. .),.,.,MOTO-SD,MOTO,SD
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,False,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,False,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,SD
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,False,NaN,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,False,NaN,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,NaN,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,NaN,14,Point (. .),.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
621,2021-0023,1,2021-03-01,2021,3,1,09:20:00,9,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,False,NaN,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS


No se tiene información que permita completar con un dato concreto estos datos faltantes, pero no se considera conveniente eliminar los registros. Por lo tanto, se decide imputar a los nulos como 'SD' indicando que es Sin Dato.

In [190]:
homicidiosHechosDf['dirección_normalizada'] = \
    homicidiosHechosDf['dirección_normalizada'].fillna('SD')
homicidiosHechosDf[homicidiosHechosDf['dirección_normalizada'] == 'SD']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,calle,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,13,Point (. .),.,.,MOTO-SD,MOTO,SD
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,False,SD,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,False,SD,0,Point (. .),.,.,PEATON-SD,PEATON,SD
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,False,SD,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,False,SD,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,14,Point (. .),.,.,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,14,Point (. .),.,.,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO
621,2021-0023,1,2021-03-01,2021,3,1,09:20:00,9,"AU BUENOS AIRES LA PLATA KM 4,5",AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,False,SD,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS


Revisamos de nuevo el tipo de dato y vemos que ya no tenemos datos nulos que puedan entorpecer el análisis.

In [191]:
dataType(homicidiosHechosDf)

,name,data_type,not_null_%,null_%,null
0,id_hecho,[<class 'str'>],100.0,0.0,0
1,n_victimas,[<class 'int'>],100.0,0.0,0
2,fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
3,aaaa,[<class 'int'>],100.0,0.0,0
4,mm,[<class 'int'>],100.0,0.0,0
5,dd,[<class 'int'>],100.0,0.0,0
6,hora,"[<class 'datetime.time'>, <class 'str'>, <clas...",100.0,0.0,0
7,hh,"[<class 'int'>, <class 'str'>]",100.0,0.0,0
8,lugar_del_hecho,[<class 'str'>],100.0,0.0,0
9,tipo_de_calle,[<class 'str'>],100.0,0.0,0


<h4>Tipos de Dato

Pasando ahora a los tipos de dato llaman la atención las columnas de hora y hh por lo que se decide normalizar. Empezando por la columna de 'hora' primero verificamos los datos que esta contiene.

In [192]:
homicidiosHechosDf['hora'].apply(type).value_counts()

hora
<class 'datetime.time'>        608
<class 'str'>                   85
<class 'datetime.datetime'>      3
Name: count, dtype: int64

Hay 88 datos que no corresponden al tipo de dato correcto, lo cuál representa el 12.64% de los datos. De estos, 85 son de tipo string y 3 de tipo datetime.

In [193]:
homicidiosHechosDf[
    homicidiosHechosDf['hora'].apply(type) == str
]['hora'].unique()

array(['22:50:00', '16:40:00', '10:12:00', '09:40:00', '22:30:00',
       '21:15:00', '02:30:00', '17:50:00', '10:30:00', '13:30:00',
       '08:15:00', '20:00:00', '01:00:00', '10:15:00', '04:15:00',
       '07:15:00', '12:30:00', '22:00:00', '10:45:00', '00:00:00',
       '06:00:00', '23:00:00', '04:45:00', '15:45:00', '18:00:00',
       '01:30:00', '13:00:00', '18:15:00', '16:45:00', '23:15:00',
       '06:30:00', '09:00:00', '07:00:00', '03:15:00', '18:45:00',
       '20:30:00', '11:15:00', '19:20:00', '17:56:00', '10:34:00',
       '08:20:00', '08:41:00', '05:35:00', '00:59:00', '22:20:00',
       '05:00:00', '14:25:00', '00:15:00', '00:30:00', '17:51:00',
       '18:12:00', '22:02:00', '14:20:00', '22:12:00', '03:05:00',
       '12:13:00', '09:18:00', '11:00:00', '02:15:00', '14:49:00',
       '13:34:00', '16:32:00', '03:32:00', '19:40:00', '21:12:00',
       '09:14:00', '04:30:00', '04:01:00', '03:00:00', '08:26:00',
       '14:07:00', '21:00:00', '15:42:00', '09:30:00', '05:25:

Convertimos los datos a formato time y manentenemos el dato 'SD' como nulo imputado.

In [194]:
homicidiosHechosDf['hora'] = pd.to_datetime(
    homicidiosHechosDf['hora'],
    format='%H:%M:%S',
    errors='coerce'
).dt.time.fillna('SD')

Ahora pasamos a la columna de 'hh' viendo los datos de tipo string.

In [195]:
homicidiosHechosDf[
    homicidiosHechosDf['hora'].apply(type) == str
]['hora'].unique()

array(['SD'], dtype=object)

Vemos que corresponde al mismo hecho imputado anteriormente, por lo que se decide mantenerlo así.

In [196]:
homicidiosHechosDf[homicidiosHechosDf['hora'].apply(type) == str]

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,calle,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
518,2019-0103,1,2019-12-18,2019,12,18,SD,SD,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


<h4>Categorías

Hay 3 variables que se clasifican en categorías: 'tipo_de_calle', 'victima', y 'acusado'. A continuación se verifica que los datos contenidos en estas columnas pertenezcan a las categorías mencionadas en el diccionario.

Para la columna de 'tipo_de_calle' y 'acusado' no se encuentran anomalías.

In [197]:
homicidiosHechosDf['tipo_de_calle'].unique()

array(['AVENIDA', 'GRAL PAZ', 'CALLE', 'AUTOPISTA'], dtype=object)

In [198]:
homicidiosHechosDf['acusado'].unique()

array(['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO',
       'MULTIPLE', 'OTRO', 'BICICLETA', 'TREN'], dtype=object)

Sin embargo, para la columna 'victima' tenemos los datos de 'OBJETO FIJO' y 'PEATON_MOTO' que no corresponden a las variables asignadas en el diccionario.

In [199]:
homicidiosHechosDf['victima'].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL', 'OBJETO FIJO', 'PEATON_MOTO'], dtype=object)

Procedemos a revisarlas más a detalle empezando por 'OBJETO FIJO'.

In [200]:
homicidiosHechosDf[homicidiosHechosDf['victima'] == 'OBJETO FIJO']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,calle,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,OBJETO FIJO,AUTO


En este caso la columna 'participantes' nos indica que las columnas 'victima' y 'acusado' están invertidas, por lo que se decide corregirlas.

In [201]:
# Se selecciona la fila usando el 'id_hecho' y se cambian los valores.
homicidiosHechosDf.loc[
    homicidiosHechosDf['id_hecho'] == '2017-0108',
    ['victima', 'acusado']
] = ['AUTO', 'OBJETO FIJO']
homicidiosHechosDf[homicidiosHechosDf['id_hecho'] == '2017-0108']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,calle,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
230,2017-0108,2,2017-09-02,2017,9,2,04:53:08,4,AV. GRAL. PAZ Y MACHAIN,GRAL PAZ,"PAZ, GRAL. AV.",True,"PAZ, GRAL. AV. y MACHAIN",12,Point (97098.48468623 109019.96106626),-58.49491054,-34.54795581,AUTO-OBJETO FIJO,AUTO,OBJETO FIJO


Ahora revisamos el caso de 'PEATON_MOTO'. Podemos observar que se trata de otro error en la captura de los datos, por lo que se decide corregir la información. En este caso no tenemos manera de inferir si la víctima se trataba de un peatón o una moto por lo que se imputa con 'SD'.

In [202]:
homicidiosHechosDf[homicidiosHechosDf['victima'] == 'PEATON_MOTO']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,calle,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,True,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,PEATON_MOTO-MOTO,PEATON_MOTO,MOTO


In [203]:
# Se selecciona la fila usando el 'id_hecho' y se cambian los valores.
homicidiosHechosDf.loc[
    homicidiosHechosDf['id_hecho'] == '2020-0063',
    ['participantes', 'victima']
] = ['SD-MOTO', 'SD']
homicidiosHechosDf.loc[homicidiosHechosDf['id_hecho'] == '2020-0063']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,calle,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
583,2020-0063,2,2020-12-05,2020,12,5,07:10:00,7,NUEVA YORK Y ALTA GRACIA,CALLE,NUEVA YORK,True,NUEVA YORK y ALTA GRACIA,11,Point (94080.62190808 102083.62453795),-58.52783814,-34.61047001,SD-MOTO,SD,MOTO


Pasando a la columna de 'acusado' tenemos que la categoría 'MULTIPLE' no corresponde a las mencionadas en el diccinario, por lo que se decide revisar a fondo.

In [204]:
homicidiosHechosDf['acusado'].unique()

array(['AUTO', 'PASAJEROS', 'SD', 'OBJETO FIJO', 'CARGAS', 'MOTO',
       'MULTIPLE', 'OTRO', 'BICICLETA', 'TREN'], dtype=object)

<h4>Datos Geográficos

Por último pasamos a verificar los datos geográficos, ya que formarán una parte importante del análisis de siniestros. Durante los análisis previos, se observaron varias instancias donde las columnas de 'xy_(caba)', 'pos_x', y 'pos_y' presentaban datos atípicos.

In [205]:
# Verificamos filas con datos atípicos en la columna 'xy_(caba)'
homicidiosHechosDf[
    homicidiosHechosDf['xy_(caba)'] == 'Point (. .)'
]['xy_(caba)'].count()

14

In [206]:
# Verificamos filas con datos atípicos en la columna 'pos_x'
homicidiosHechosDf[homicidiosHechosDf['pos_x'] == '.']['pos_x'].count()

12

In [207]:
# Verificamos filas con datos atípicos en la columna 'pos_x'
homicidiosHechosDf[homicidiosHechosDf['pos_y'] == '.']['pos_y'].count()

12

In [208]:
homicidiosHechosDf[homicidiosHechosDf['xy_(caba)'] == 'Point (. .)']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,calle,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
35,2016-0049,1,2016-04-17,2016,4,17,00:00:00,0,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI KM....,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,False,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,SD,SD
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,13,Point (. .),.,.,MOTO-SD,MOTO,SD
71,2016-0096,1,2016-07-25,2016,7,25,07:00:00,7,"AUTOPISTA DELLEPIANE LUIS TTE. GRAL. KM. 2,3",AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,False,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,8,Point (. .),-58.47433193007387,-34.66684950051973,MOTO-CARGAS,MOTO,CARGAS
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,False,SD,4,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,False,SD,0,Point (. .),.,.,PEATON-SD,PEATON,SD
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,False,AUTOPISTA 25 DE MAYO,0,Point (. .),.,.,SD-SD,SD,SD
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",False,"LUGONES, LEOPOLDO AV.",14,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,False,SD,9,Point (. .),.,.,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,False,SD,7,Point (. .),.,.,AUTO-AUTO,AUTO,AUTO
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,False,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,Point (. .),.,.,MOTO-PASAJEROS,MOTO,PASAJEROS


Utilizando la [API](https://datosabiertos-apis.buenosaires.gob.ar/BA_Root/Documentacion?schema_name=usig_geocoder) del gobierno de Buenos Aires, encontramos los datos de 'xy_(caba)' para los hechos '2016-0049' y '2016-0096' por lo que se añaden. El resto de datos se imputan con 'SD'.

In [209]:
# Corregimos el ID 2016-0049
homicidiosHechosDf.loc[
    homicidiosHechosDf['id_hecho'] == '2016-0049',
    ['xy_(caba)']
] = ['Point (107929.142708 99229.9028876)']
# Corregimos el ID 2016-0096
homicidiosHechosDf.loc[
    homicidiosHechosDf['id_hecho'] == '2016-0096',
    ['xy_(caba)']
] = ['Point (98974.6284645 95927.1486379)']
# Imputamos con 'SD'
homicidiosHechosDf.loc[
    homicidiosHechosDf['pos_x'] == '.',
    ['xy_(caba)','pos_x','pos_y']
] = ['SD','SD','SD']
homicidiosHechosDf[homicidiosHechosDf['pos_x'] == 'SD']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,calle,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado
38,2016-0052,1,2016-04-20,2016,4,20,20:00:00,20,AUTOPISTA LUGONES PK 10000,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,13,SD,SD,SD,MOTO-SD,MOTO,SD
106,2016-0136,1,2016-10-25,2016,10,25,00:00:00,0,AU BUENOS AIRES - LA PLATA KM. 4,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,False,SD,4,SD,SD,SD,MOTO-CARGAS,MOTO,CARGAS
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,SD,CALLE,SD,False,SD,0,SD,SD,SD,PEATON-SD,PEATON,SD
139,2016-0174,1,2016-12-27,2016,12,27,00:00:00,0,AUTOPISTA 25 DE MAYO,AUTOPISTA,AUTOPISTA 25 DE MAYO,False,AUTOPISTA 25 DE MAYO,0,SD,SD,SD,SD-SD,SD,SD
176,2017-0042,1,2017-04-10,2017,4,10,09:00:00,9,AV. LEOPOLDO LUGONES PKM 6900,GRAL PAZ,"LUGONES, LEOPOLDO AV.",False,"LUGONES, LEOPOLDO AV.",14,SD,SD,SD,MOTO-CARGAS,MOTO,CARGAS
180,2017-0050,2,2017-04-28,2017,4,28,11:08:08,11,AU PERITO MORENO Y RAMAL ENLACE AU1/AU6,AUTOPISTA,AUTOPISTA PERITO MORENO,False,SD,9,SD,SD,SD,MOTO-CARGAS,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,1,03:47:47,3,AU DELLEPIANE 2400,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,False,SD,7,SD,SD,SD,AUTO-AUTO,AUTO,AUTO
256,2017-0140,1,2017-11-19,2017,11,19,23:22:17,23,AU ARTURO FRONDIZI PKM 3100,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,False,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,4,SD,SD,SD,MOTO-PASAJEROS,MOTO,PASAJEROS
313,2018-0039,1,2018-04-21,2018,4,21,22:15:00,22,AUTOPISTA LUGONES KM 4.7,AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,14,SD,SD,SD,PEATON-AUTO,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,17,06:40:00,6,"LUGONES, LEOPOLDO AV. KM 6,1",AUTOPISTA,"LUGONES, LEOPOLDO AV.",False,SD,14,SD,SD,SD,MOTO-OBJETO FIJO,MOTO,OBJETO FIJO


Por último agregamos la columna 'day_name' para facilitar el análisis posterior.

In [210]:
homicidiosHechosDf['day_name'] = pd.to_datetime(homicidiosHechosDf['fecha'])
homicidiosHechosDf['day_name'] = homicidiosHechosDf['day_name'].dt.day_of_week
homicidiosHechosDf['day_name'] = homicidiosHechosDf['day_name'].replace({0:'Lunes',1:'Martes',2:'Miércoles',3:'Jueves',4:'Viernes',5:'Sábado',6:'Domingo'})

In [211]:
homicidiosHechosDf.head()

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,cruce,dirección_normalizada,comuna,xy_(caba),pos_x,pos_y,participantes,victima,acusado,day_name
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,True,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,Viernes
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,True,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,False,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,Domingo
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,True,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD,Domingo
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,True,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves


<h2>Homicidios - Víctimas

<h3>Revisión Inicial

Se carga el dataset y se crea el dataframe correspondiente.

In [212]:
homicidiosVictimasDf = pd.read_excel(
    '../data/homicidios.xlsx',
    sheet_name = 'VICTIMAS'
)

Comenzamos visualizando el dataframe de víctimas, vemos que cuenta con 717 registros, llama la atención que son más registros de víctimas que de hechos. En primera instancia ya se pueden observar datos donde será necesario realizar transformación.

In [213]:
homicidiosVictimasDf.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


In [214]:
homicidiosVictimasDf.shape[0]

717

Normalizamos los nombres de las columnas.

In [215]:
homicidiosVictimasDf.columns = [x.lower() for x in homicidiosVictimasDf.columns]
homicidiosVictimasDf.columns = homicidiosVictimasDf.columns.str.replace(
    ' ', 
    '_'
)
homicidiosVictimasDf.columns

Index(['id_hecho', 'fecha', 'aaaa', 'mm', 'dd', 'rol', 'victima', 'sexo',
       'edad', 'fecha_fallecimiento'],
      dtype='object')

<h3>Revisión de Datos

<h4>Datos Duplicados

Comprobamos la existencia de filas duplicadas, especialmente en la columna de 'id_hecho'.

In [216]:
homicidiosVictimasDf[homicidiosVictimasDf.duplicated()]

,id_hecho,fecha,aaaa,mm,dd,rol,victima,sexo,edad,fecha_fallecimiento


Se encuentran múltiples datos duplicados en la columna de 'id_hecho', sin embargo, estos representan el hecho de que existe múltiples víctimas por siniestro, por lo que se conservan las filas.

In [217]:
homicidiosVictimasDf[homicidiosVictimasDf.duplicated(subset=['id_hecho'], keep=False)]

,id_hecho,fecha,aaaa,mm,dd,rol,victima,sexo,edad,fecha_fallecimiento
29,2016-0041,2016-03-29,2016,3,29,CONDUCTOR,MOTO,MASCULINO,54,2016-03-29 00:00:00
30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,SD,2016-03-30 00:00:00
98,2016-0126,2016-09-18,2016,9,18,CONDUCTOR,AUTO,MASCULINO,37,SD
99,2016-0126,2016-09-18,2016,9,18,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,60,SD
163,2017-0026,2017-02-26,2017,2,26,PASAJERO_ACOMPAÑANTE,AUTO,FEMENINO,23,2017-02-26 00:00:00
164,2017-0026,2017-02-26,2017,2,26,CONDUCTOR,AUTO,MASCULINO,19,2017-02-26 00:00:00
173,2017-0035,2017-03-23,2017,3,23,CONDUCTOR,AUTO,MASCULINO,28,2017-03-23 00:00:00
174,2017-0035,2017-03-23,2017,3,23,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,32,2017-03-23 00:00:00
175,2017-0035,2017-03-23,2017,3,23,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,30,2017-03-23 00:00:00
176,2017-0036,2017-03-29,2017,3,29,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,SD,2017-03-29 00:00:00


<h4>Datos Nulos

Aplicamos la función dataType para determinar la cantidad de valores nulos.

In [218]:
dataType(homicidiosVictimasDf)

,name,data_type,not_null_%,null_%,null
0,id_hecho,[<class 'str'>],100.0,0.0,0
1,fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
2,aaaa,[<class 'int'>],100.0,0.0,0
3,mm,[<class 'int'>],100.0,0.0,0
4,dd,[<class 'int'>],100.0,0.0,0
5,rol,[<class 'str'>],100.0,0.0,0
6,victima,[<class 'str'>],100.0,0.0,0
7,sexo,[<class 'str'>],100.0,0.0,0
8,edad,"[<class 'int'>, <class 'str'>]",100.0,0.0,0
9,fecha_fallecimiento,"[<class 'datetime.datetime'>, <class 'str'>]",100.0,0.0,0


Vemos que no existen datos nulos por lo que pasamos a revisar el tipo de dato.

<h4>Tipos de Dato

Empezando por la columna de 'edad', esta hace sentido que esté compuesta de integer, por lo que se revisan las instancias donde el dato es de tipo string.

In [219]:
homicidiosVictimasDf['edad'].apply(type).value_counts()

edad
<class 'int'>    664
<class 'str'>     53
Name: count, dtype: int64

Vemos que hay 53 datos como string, lo cual representa un 7.3% de los datos. Al revisar detalladamente podemos observar que se trata de datos imputados en los que no se conoce la edad de la víctima. Siguiendo la lógica utilizada en la pestaña de hechos, ya que se considera la cantidad de víctimas que no pudieron ser identificadas como un dato importante en sí mismo se decide mantenerlos.

In [220]:
homicidiosVictimasDf[homicidiosVictimasDf['edad'].apply(type) == str]

,id_hecho,fecha,aaaa,mm,dd,rol,victima,sexo,edad,fecha_fallecimiento
30,2016-0041,2016-03-29,2016,3,29,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,SD,2016-03-30 00:00:00
33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,SD,SD
35,2016-0048,2016-04-15,2016,4,15,PEATON,PEATON,FEMENINO,SD,SD
36,2016-0049,2016-04-17,2016,4,17,SD,SD,SD,SD,SD
39,2016-0052,2016-04-20,2016,4,20,SD,MOTO,SD,SD,SD
55,2016-0077,2016-06-13,2016,6,13,PEATON,PEATON,FEMENINO,SD,SD
63,2016-0085,2016-06-29,2016,6,29,SD,MOTO,MASCULINO,SD,SD
72,2016-0096,2016-07-25,2016,7,25,CONDUCTOR,MOTO,MASCULINO,SD,SD
89,2016-0115,2016-09-02,2016,9,2,SD,SD,MASCULINO,SD,SD
93,2016-0119,2016-09-04,2016,9,4,PASAJERO_ACOMPAÑANTE,SD,FEMENINO,SD,SD


Pasamos a revisar la columna de 'fecha_fallecimiento' que igualmente contiene datos de tipo string.

In [221]:
homicidiosVictimasDf['fecha_fallecimiento'].apply(type).value_counts()

fecha_fallecimiento
<class 'datetime.datetime'>    648
<class 'str'>                   69
Name: count, dtype: int64

Observamos que 69 de los datos son de tipo string, lo cual representa un 9.6% de los datos. Pasamos a revisarlo más detenidamente,

In [222]:
homicidiosVictimasDf[
    (homicidiosVictimasDf['fecha_fallecimiento'].apply(type) == str)
]

,id_hecho,fecha,aaaa,mm,dd,rol,victima,sexo,edad,fecha_fallecimiento
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
16,2016-0022,2016-02-21,2016,2,21,PASAJERO_ACOMPAÑANTE,MOTO,MASCULINO,41,SD
19,2016-0027,2016-02-28,2016,2,28,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,34,SD
22,2016-0031,2016-03-08,2016,3,8,CONDUCTOR,MOTO,MASCULINO,21,SD
33,2016-0045,2016-04-11,2016,4,11,CONDUCTOR,MOTO,MASCULINO,SD,SD
...,...,...,...,...,...,...,...,...,...,...
242,2017-0112,2017-09-10,2017,9,10,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,1,SD
246,2017-0115,2017-09-19,2017,9,19,CONDUCTOR,MOTO,MASCULINO,34,SD
253,2017-0126,2017-10-14,2017,10,14,PASAJERO_ACOMPAÑANTE,AUTO,MASCULINO,39,SD
280,2017-0155,2017-12-12,2017,12,12,SD,SD,MASCULINO,77,SD


Vemos que muchas de las columnas están compuestas por 'SD', realizamos la consulta ignorando estas filas ya que no contamos con la información necesaria para completarlas, sin embargo, se consideran relevantes para el análisis.

In [223]:
homicidiosVictimasDf[
    (homicidiosVictimasDf['fecha_fallecimiento'].apply(type) == str) & 
    (homicidiosVictimasDf['fecha_fallecimiento'] != 'SD')
]

,id_hecho,fecha,aaaa,mm,dd,rol,victima,sexo,edad,fecha_fallecimiento
454,2019-0019,2019-03-06,2019,3,6,CONDUCTOR,MOTO,MASCULINO,SD,26/03/2019


Transformamos toda la columna a formato AAAA-MM-DD y mantenemos las filas sin datos como 'SD'.

In [224]:
homicidiosVictimasDf['fecha_fallecimiento'] = pd.to_datetime(
    homicidiosVictimasDf['fecha_fallecimiento'],
    errors='coerce'
).dt.date.fillna('SD')


Revisamos que se hayan realizado los cambios correctamente.

In [225]:
homicidiosVictimasDf[homicidiosVictimasDf['id_hecho'] == '2019-0019']

,id_hecho,fecha,aaaa,mm,dd,rol,victima,sexo,edad,fecha_fallecimiento
454,2019-0019,2019-03-06,2019,3,6,CONDUCTOR,MOTO,MASCULINO,SD,2019-03-26


In [226]:
homicidiosVictimasDf.head()

,id_hecho,fecha,aaaa,mm,dd,rol,victima,sexo,edad,fecha_fallecimiento
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01


<h4>Categorías

A continuación se verifica que los datos contenidos en la columna 'victima' pertenezcan a las categorías mencionadas en el diccionario y vemos que no hay anomalías.

In [227]:
homicidiosVictimasDf['victima'].unique()

array(['MOTO', 'AUTO', 'PEATON', 'SD', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL'], dtype=object)

Por último se guardan ambos dataframe en formato csv.

In [228]:
homicidiosHechosDf.to_csv(
    '../data/homicidios_hechos.csv',
    encoding='utf-8',
    index=False
)
homicidiosVictimasDf.to_csv(
    '../data/homicidios_victimas.csv',
    encoding='utf-8', 
    index=False
)

<h2>Preparación Power BI

Como parte del proyecto se nos solicita crear un dashboard en Power BI, para esto se crea un dataframe uniendo la información de los datasets trabajados en este notebook. Como parte de la preparación para trabajar los datos en Power BI se penalizan los datos faltantes de las columnas numéricas con la media absoluta de la columna a la cual pertenecen. Esto con el fin de mantener un solo tipo de dato por columna. Los campos sin datos de las columnas alfanuméricas se mantienen con 'SD'.

In [229]:
homicidiosDf = pd.merge(
    homicidiosHechosDf,
    homicidiosVictimasDf[
        ['id_hecho', 'rol', 'sexo', 'edad', 'fecha_fallecimiento']
    ],
    how='right',
    on='id_hecho'
)

In [230]:
homicidiosDf.head()

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,pos_x,pos_y,participantes,victima,acusado,day_name,rol,sexo,edad,fecha_fallecimiento
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO,Viernes,CONDUCTOR,MASCULINO,19,2016-01-01
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS,Sábado,CONDUCTOR,MASCULINO,70,2016-01-02
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO,Domingo,CONDUCTOR,MASCULINO,30,2016-01-03
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,-58.46503904,-34.68092974,MOTO-SD,MOTO,SD,Domingo,CONDUCTOR,MASCULINO,18,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS,Jueves,CONDUCTOR,MASCULINO,29,2016-02-01


Aplicamos la función dataType para identificar las columnas que requieren atención. En este caso son las de 'hora', 'hh', 'edad', y 'fecha_fallecimiento'.

In [231]:
dataType(homicidiosDf)

,name,data_type,not_null_%,null_%,null
0,id_hecho,[<class 'str'>],100.0,0.0,0
1,n_victimas,[<class 'int'>],100.0,0.0,0
2,fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
3,aaaa,[<class 'int'>],100.0,0.0,0
4,mm,[<class 'int'>],100.0,0.0,0
5,dd,[<class 'int'>],100.0,0.0,0
6,hora,"[<class 'datetime.time'>, <class 'str'>]",100.0,0.0,0
7,hh,"[<class 'int'>, <class 'str'>]",100.0,0.0,0
8,lugar_del_hecho,[<class 'str'>],100.0,0.0,0
9,tipo_de_calle,[<class 'str'>],100.0,0.0,0


Obtenemos la media de las columnas seleccionadas.

In [232]:
meanHora = pd.to_datetime(
    homicidiosDf['hora'],
    format='%H:%M:%S',
    errors='coerce'
).dropna().mean()

homicidiosDf['hora'] = homicidiosDf['hora'].replace({'SD':meanHora.time()})

homicidiosDf['hora'].fillna(meanHora, inplace=True)

homicidiosDf['hh'] = pd.to_numeric(homicidiosDf['hh'], errors='coerce')
meanHH = homicidiosDf['hh'].dropna().mean()
homicidiosDf['hh'].fillna(meanHH, inplace=True)

homicidiosDf['edad'] = pd.to_numeric(homicidiosDf['edad'], errors='coerce')
meanEdad = homicidiosDf['edad'].dropna().mean()
homicidiosDf['edad'].fillna(meanEdad, inplace=True)

In [234]:
dataType(homicidiosDf)

,name,data_type,not_null_%,null_%,null
0,id_hecho,[<class 'str'>],100.0,0.0,0
1,n_victimas,[<class 'int'>],100.0,0.0,0
2,fecha,[<class 'pandas._libs.tslibs.timestamps.Timest...,100.0,0.0,0
3,aaaa,[<class 'int'>],100.0,0.0,0
4,mm,[<class 'int'>],100.0,0.0,0
5,dd,[<class 'int'>],100.0,0.0,0
6,hora,[<class 'datetime.time'>],100.0,0.0,0
7,hh,[<class 'float'>],100.0,0.0,0
8,lugar_del_hecho,[<class 'str'>],100.0,0.0,0
9,tipo_de_calle,[<class 'str'>],100.0,0.0,0


In [235]:
homicidiosDf[homicidiosDf['pos_x'] == '-58.39897064']

,id_hecho,n_victimas,fecha,aaaa,mm,dd,hora,hh,lugar_del_hecho,tipo_de_calle,...,pos_x,pos_y,participantes,victima,acusado,day_name,rol,sexo,edad,fecha_fallecimiento
357,2018-0069,1,2018-07-04,2018,7,4,21:45:00,21.0,Córdoba Av. 2215\n\n,AVENIDA,...,-58.39897064,-34.59954925,BICICLETA-PASAJEROS,BICICLETA,PASAJEROS,Miércoles,CICLISTA,MASCULINO,20.0,2018-07-04


In [236]:
homicidiosDf.to_csv(
    '../data/homicidios_merge.csv',
    encoding='utf-8',
    index=False
)